# sql_analysis.ipynb
----

Written in the Python 3.7.9 Environment

By Nicole Lund 

This Jupyter Notebook analyzes the postgreSQL database.

The queries containe herin were developed in sql.sql and transformed into sqlalchemy.

In [68]:
# Import Dependencies
import sys
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# Import database password
try:
    from postgres_pswd import host, database, username, passwd
    if host == 'YOUR DATABASE HOST HERE':
        sys.path.append(r"C:\Users\nlund\Documents\GitHub\untracked_files")
        from postgres_remote import host, database, username, passwd
except:
    sys.path.append(r"C:\Users\nlund\Documents\GitHub\untracked_files")
    from postgres_remote import host, database, username, passwd

# Create engine to mutual_funds database
engine_startup = 'postgresql://' + username + ":" + passwd + "@" + host + '/' + database
engine = create_engine(engine_startup)

In [14]:
# reflect the existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# View all of the classes that automap found
Base.classes.keys()

['sp500', 'fund_holdings']

In [24]:
# Save references to each table
sp500 = Base.classes.sp500
holdings = Base.classes.fund_holdings

In [25]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [29]:
# Collect Total Market Value for each fund
market_total = session.query(holdings.fund_name, func.sum(holdings.market_value)).group_by(holdings.fund_name).all()
market_total

[('Voya LargeCap Growth Portfolio', Decimal('6532218897.430000055')),
 ('Voya Russell LargeCap Index Portfolio', Decimal('1244100190.319999995')),
 ('Voya U.S. Stock Index Portfolio', Decimal('7917050279.339999898')),
 ('Voya LargeCap Value Portfolio', Decimal('1113767104.140000002')),
 ('Voya Index Plus LargeCap Portfolio', Decimal('959911558.350000009'))]

In [79]:
# Collect sp500 holdings market value for each fund
query = session.query(holdings).join(sp500, holdings.ticker==sp500.ticker)
for row in query.all():
    print (row)

InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: SELECT fund_holdings.index AS fund_holdings_index, fund_holdings.fund_name AS fund_holdings_fund_name, fund_holdings.ticker AS fund_holdings_ticker, fund_holdings.security_name AS fund_holdings_security_name, fund_holdings.currency AS fund_holdings_currency, fund_holdings.country AS fund_holdings_country, fund_holdings.price AS fund_holdings_price, fund_holdings.quantity AS fund_holdings_quantity, fund_holdings.market_value AS fund_holdings_market_value 
FROM fund_holdings JOIN sp500 ON fund_holdings.ticker = sp500.ticker]
(Background on this error at: http://sqlalche.me/e/13/2j85)

In [ ]:
# Close Session
session.close()